# Restaurant Wait Decision - From Simple to Complex Trees

## Introduction

We’ll explore how decision trees can predict whether customers will wait
for a table, starting with simple single-feature trees and building up
to more complex models. This progression will help us understand how
different features contribute to the prediction.

## Setting Up Our Helper Functions

First, we’ll create some helper functions to make our trees easier to
read and understand:

In [ ]:
def clean_tree_text(tree_text):
    # Dictionary of all replacements
    replacements = {
        "patrons_Full <= 0.5": "Patrons not Full?",
        "patrons_None <= 0.5": "Some or Full Patrons?",
        "patrons_Some <= 0.5": "None or Full Patrons?",
        "price_Cheap <= 0.5": "Moderate or Expensive?",
        "price_Expensive <= 0.5": "Cheap or Moderate?",
        "price_Moderate <= 0.5": "Cheap or Expensive?",
        "type_Burger <= 0.5": "Not Burger?",
        "type_French <= 0.5": "Not French?",
        "type_Italian <= 0.5": "Not Italian?",
        "type_Thai <= 0.5": "Not Thai?",
        "waitestimate_0-10 <= 0.5": "Wait > 10min?",
        "waitestimate_10-30 <= 0.5": "Wait not 10-30min?",
        "waitestimate_30-60 <= 0.5": "Wait not 30-60min?",
        "waitestimate_>60 <= 0.5": "Wait not >60min?",
        "alternate_yes <= 0.5": "No Alternative?",
        "bar_yes <= 0.5": "No Bar?",
        "fri/sat_yes <= 0.5": "Not Weekend?",
        "hungry_yes <= 0.5": "Not Hungry?",
        "raining_yes <= 0.5": "Not Raining?",
        "reservation_yes <= 0.5": "No Reservation?",
        "class:": ""
    }
    
    # Apply all replacements
    clean_text = tree_text
    for old, new in replacements.items():
        clean_text = clean_text.replace(old, new)
    
    return clean_text

def print_clean_tree(classifier, columns):
    tree_text = export_text(classifier, feature_names=list(columns))
    print("\nDecision Tree:")
    print(clean_tree_text(tree_text))

def print_accuracy_metrics(predictions, actuals):
    cm = confusion_matrix(actuals, predictions)
    cm_df = pd.DataFrame(
        cm,
        index=['Actual: Won\'t Wait', 'Actual: Will Wait'],
        columns=['Predicted: Won\'t Wait', 'Predicted: Will Wait']
    )
    print("\nConfusion Matrix:")
    print(cm_df)
    
    correct = cm[0,0] + cm[1,1]
    total = cm[0,0] + cm[1,1] + cm[1,0] + cm[0,1]
    accuracy = correct/total
    print(f"\nAccuracy: {accuracy:.2%}")

## The Data

Let’s load our training and testing data:

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import confusion_matrix

# Training data
train = pd.DataFrame({
    'alternate': ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes'],
    'bar': ['No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'fri/sat': ['No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'hungry': ['Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
    'patrons': ['Some', 'Full', 'Some', 'Full', 'Full', 'Some', 'None', 'Some', 'Full', 'Full', 'None', 'Full'],
    'price': ['Expensive', 'Cheap', 'Cheap', 'Cheap', 'Expensive', 'Moderate', 'Cheap', 'Moderate', 'Cheap', 'Expensive', 'Cheap', 'Cheap'],
    'raining': ['No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No'],
    'reservation': ['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No'],
    'type': ['French', 'Thai', 'Burger', 'Thai', 'French', 'Italian', 'Burger', 'Thai', 'Burger', 'Italian', 'Thai', 'Burger'],
    'waitestimate': ['0-10', '30-60', '0-10', '10-30', '>60', '0-10', '0-10', '0-10', '>60', '10-30', '0-10', '30-60'],
    'willwait': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes']
})

# Testing data
test = pd.DataFrame({
    'alternate': ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes'],
    'bar': ['Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No'],
    'fri/sat': ['No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No'],
    'hungry': ['Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No'],
    'patrons': ['Full', 'Full', 'Some', 'None', 'Some', 'Full', 'Full'],
    'price': ['Expensive', 'Moderate', 'Expensive', 'Cheap', 'Cheap', 'Expensive', 'Cheap'],
    'raining': ['Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes'],
    'reservation': ['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No'],
    'type': ['Thai', 'Italian', 'French', 'Burger', 'Thai', 'Italian', 'Burger'],
    'waitestimate': ['0-10', '0-10', '30-60', '0-10', '10-30', '>60', '0-10'],
    'willwait': ['No', 'No', 'Yes', 'No', 'Yes', 'No', 'No']
})

print("Training Data:")
print(train)
print("\nTesting Data:")
print(test)

## Model 1: Single Feature - Wait Time

Let’s start with just the wait time estimate, which intuitively might be
important:

In [ ]:
# Single feature model - Wait Time
X_train = pd.get_dummies(train[['waitestimate']])
X_test = pd.get_dummies(test[['waitestimate']])

classifier = DecisionTreeClassifier()
classifier.fit(X_train, train['willwait'])
predictions = classifier.predict(X_test)

print("Decision tree using only wait time:")
print_clean_tree(classifier, X_train.columns)
print_accuracy_metrics(predictions, test['willwait'])

## Model 2: Two Key Features - Wait Time and Hunger

Now let’s add whether the customers are hungry:

In [ ]:
# Two feature model
X_train = pd.get_dummies(train[['waitestimate', 'hungry']])
X_test = pd.get_dummies(test[['waitestimate', 'hungry']])

classifier = DecisionTreeClassifier()
classifier.fit(X_train, train['willwait'])
predictions = classifier.predict(X_test)

print("Decision tree using wait time and hunger:")
print_clean_tree(classifier, X_train.columns)
print_accuracy_metrics(predictions, test['willwait'])

## Model 3: Restaurant Characteristics

Let’s try using features about the restaurant itself:

In [ ]:
# Restaurant characteristics model
X_train = pd.get_dummies(train[['price', 'type', 'patrons']])
X_test = pd.get_dummies(test[['price', 'type', 'patrons']])

classifier = DecisionTreeClassifier()
classifier.fit(X_train, train['willwait'])
predictions = classifier.predict(X_test)

print("Decision tree using restaurant characteristics:")
print_clean_tree(classifier, X_train.columns)
print_accuracy_metrics(predictions, test['willwait'])

## Model 4: Full Model

Finally, let’s use all available features:

In [ ]:
# Full model with all features
features = ['alternate', 'bar', 'fri/sat', 'hungry', 'patrons', 
            'price', 'raining', 'reservation', 'type', 'waitestimate']

X_train = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

classifier = DecisionTreeClassifier()
classifier.fit(X_train, train['willwait'])
predictions = classifier.predict(X_test)

print("Decision tree using all features:")
print_clean_tree(classifier, X_train.columns)
print_accuracy_metrics(predictions, test['willwait'])

## Understanding the Results

Let’s analyze what we learned from each model:

1.  Wait Time Only Model:
    -   Shows basic intuition about patience
    -   Limited predictive power since same wait times can lead to
        different decisions
2.  Wait Time and Hunger Model:
    -   Adds customer motivation
    -   Shows how hunger can override wait time concerns
    -   Still misses important context
3.  Restaurant Characteristics Model:
    -   Captures effects of price, type, and crowdedness
    -   Shows how restaurant attributes influence waiting decisions
    -   Missing situational factors
4.  Full Model:
    -   Combines all aspects into more nuanced predictions
    -   Can capture complex decision patterns
    -   Shows interactions between different features

Key insights: 1. Simple trees can capture basic patterns but miss
nuances 2. Adding features allows the tree to handle more complex
scenarios 3. Some combinations of features are particularly important

The confusion matrices show: - How accurately each model predicts both
waiting and not waiting - Where each model tends to make mistakes - How
accuracy improves with more features

This analysis could help restaurants: - Better understand what
influences waiting decisions - Identify which factors matter most - Make
better decisions about reservations and customer management